In [5]:
# Import all necessary packages

import numpy as np 
import pandas as pd 
import json
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
from datetime import datetime
import glob
import seaborn as sns
import re
import os
import nltk
#nltk.download()
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk import sent_tokenize, word_tokenize

from collections import Counter
from nltk.tokenize import RegexpTokenizer
import re
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/jean/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
# Loading the dataset

my_df = pd.read_csv('/home/jean/4549_466349_bundle_archive/FRvideos.csv', index_col='video_id')
my_df.head()

,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
video_id,,,,,,,,,,,,,,,
Ro6eob0LrCY,17.14.11,Malika LePen : Femme de Gauche - Trailer,Le Raptor Dissident,24,2017-11-13T17:32:55.000Z,"Raptor""|""Dissident""|""Expliquez""|""moi""|""cette""|...",212702,29282,1108,3817,https://i.ytimg.com/vi/Ro6eob0LrCY/default.jpg,False,False,False,Dimanche.\n18h30.\nSoyez présents pour la vidé...
Yo84eqYwP98,17.14.11,"LA PIRE PARTIE ft Le Rire Jaune, Pierre Croce,...",Le Labo,24,2017-11-12T15:00:02.000Z,[none],432721,14053,576,1161,https://i.ytimg.com/vi/Yo84eqYwP98/default.jpg,False,False,False,Le jeu de société: https://goo.gl/hhG1Ta\n\nGa...
ceqntSXE-10,17.14.11,DESSINS ANIMÉS FRANÇAIS VS RUSSES 2 - Daniil...,Daniil le Russe,23,2017-11-13T17:00:38.000Z,"cartoon""|""pokémon""|""école""|""ours""|""мультфильм",482153,76203,477,9580,https://i.ytimg.com/vi/ceqntSXE-10/default.jpg,False,False,False,Une nouvelle dose de dessins animés français e...
WuTFI5qftCE,17.14.11,PAPY GRENIER - METAL GEAR SOLID,Joueur Du Grenier,20,2017-11-12T17:00:02.000Z,"Papy grenier""|""Metal Gear Solid""|""PS1""|""Tirage...",925222,85016,550,4303,https://i.ytimg.com/vi/WuTFI5qftCE/default.jpg,False,False,False,"Nouvel ,épisode de Papy Grenier ! Ce mois-ci o..."
ee6OFs8TdEg,17.14.11,QUI SAUTERA LE PLUS HAUT ? (VÉLO SKATE ROLLER ...,Aurelien Fontenoy,17,2017-11-13T16:30:03.000Z,"vélo""|""vtt""|""bmx""|""freestyle""|""bike""|""mtb""|""di...",141695,8091,72,481,https://i.ytimg.com/vi/ee6OFs8TdEg/default.jpg,False,False,False,Sauts à plus de 4 mètres de haut dans un tramp...


In [3]:
# Laoading stopwords to remove it from the text

en_stopwords = list(stopwords.words('english'))
fr_stopwords = list(stopwords.words('french'))   
en_stopwords.extend(fr_stopwords)


In [7]:
# Filtering tags with regex to tokenized it

my_df['tags'] = my_df['tags'].str.lower()


for i in range (len(my_df['tags'])):
    tw = re.sub('[^A-Za-zçéè]+', ' ', my_df.iloc[i]['tags'])
    word_tokens = word_tokenize(tw)
    filtered_sentence = [w for w in word_tokens if not w in en_stopwords]
    without_single_chr = [word for word in filtered_sentence if len(word) > 2]
    my_df['tags'][i] = str([word for word in without_single_chr if not word.isdigit()])
 

/home/jean/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [16]:
# Dropping tags that were empty before ([none]) and tags that are empty after ([]) filtering

my_df = my_df[my_df.tags != "['none']"]
my_df = my_df[my_df.tags != "[]"]


In [7]:
# Reformate date

my_df['trending_date'] = pd.to_datetime(my_df['trending_date'],errors='coerce', format='%y.%d.%m')
my_df['publish_time'] = pd.to_datetime(my_df['publish_time'], errors='coerce', format='%Y-%m-%dT%H:%M:%S.%fZ')

my_df = my_df[my_df['trending_date'].notnull()]
my_df = my_df[my_df['publish_time'].notnull()]

my_df = my_df.dropna(how='any',inplace=False, axis = 0)

my_df.insert(4, 'publish_date', my_df['publish_time'].dt.date)
my_df['publish_time'] = my_df['publish_time'].dt.time

my_df_full = my_df.reset_index().sort_values('trending_date').set_index('video_id')
my_df = my_df.reset_index().sort_values('trending_date').drop_duplicates('video_id',keep='last').set_index('video_id')
my_df[['publish_date','publish_time']].head()

,publish_date,publish_time
video_id,,
k_KXgkEZGtI,2017-11-12,11:42:39
tgjTYiowYZk,2017-11-13,16:59:03
jKfNUQsYMYI,2017-11-11,17:00:05
1cr9mQh_IlA,2017-11-11,19:06:46
7Cj6P13MUNU,2017-11-13,08:00:00


In [9]:
# Associating category_id with the correct category name with the .json file

my_df['category_id'] = my_df['category_id'].astype(str)
my_df_full['category_id'] = my_df['category_id'].astype(str)

category_id = {}

with open('/home/jean/4549_466349_bundle_archive/FR_category_id.json', 'r') as f:
    data = json.load(f)
    for category in data['items']:
        category_id[category['id']] = category['snippet']['title']

my_df.insert(4, 'category', my_df['category_id'].map(category_id))
my_df_full.insert(4, 'category', my_df_full['category_id'].map(category_id))
category_list = my_df['category'].unique()
category_list

array(['Sports', 'Music', 'Science & Technology', 'Entertainment',
       'Howto & Style', 'Gaming', 'People & Blogs', 'News & Politics',
       'Comedy', 'Film & Animation', 'Autos & Vehicles', 'Education',
       'Travel & Events', 'Shows', 'Pets & Animals', nan, 'Movies',
       'Trailers'], dtype=object)

In [10]:
# Create new data values 'publish_to_train' and 'trend_duration' to make dataframe more explicit 

publish_to_trend = {}
my_df_first = my_df_full.reset_index().drop_duplicates('video_id',keep ='first').set_index('video_id')
diff_first = (my_df_first['trending_date']).astype('datetime64[ns]')-my_df_first['publish_date'].astype('datetime64[ns]')

diff_first = diff_first.reset_index()
diff_first.columns = ['video_id','publish_to_trend']

for i, row in diff_first.iterrows():
    publish_to_trend[row['video_id']] = row['publish_to_trend'].days

my_df_last = my_df
diff_last = my_df['trending_date'].astype('datetime64[ns]')-my_df['publish_date'].astype('datetime64[ns]')
diff_last = diff_last.reset_index()
diff_last.columns = ['video_id','publish_to_trend_last']
my_df = my_df.reset_index()
my_df.insert(4,'publish_to_trend_last', diff_last['publish_to_trend_last'].astype('timedelta64[D]').astype(int))
my_df.insert(4, 'publish_to_trend', my_df['video_id'].map(publish_to_trend))
my_df.insert(4, 'trend_duration', 0)
my_df['trend_duration'] = (my_df['publish_to_trend_last']-my_df['publish_to_trend'])+1
my_df = my_df[my_df.video_id != 'FeRi86DcfyA']
my_df.set_index('video_id')[['publish_to_trend','trend_duration']].sort_values('trend_duration',ascending=True).head()

,publish_to_trend,trend_duration
video_id,,
k_KXgkEZGtI,2,1
PKBug0-mzGs,1,1
vIc_p28woYA,1,1
qebEL68KsdY,1,1
OoREgT676E0,1,1


In [8]:
# Droping empty row

my_df = my_df.dropna(how='any',inplace=False, axis = 0)

In [22]:
# Saving the trnasformation

my_df.to_csv(r'/home/jean/clean_data.csv', index = False)